### **0. Instalacion e imports de librerias**

In [500]:
import sys
# Version 3.8 de Python requerida
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install contexto
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install openpyxl

In [501]:
from contexto.limpieza import limpieza_texto, lista_stopwords, remover_stopwords
from contexto.stemming import Stemmer, stem_texto
from contexto.vectorizacion import VectorizadorFrecuencias

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import numpy as np

### **1. Exploracion de datos**

In [502]:
df = pd.read_excel('cat_6716.xlsx')

In [503]:
df.head()

,Textos_espanol,sdg
0,"Es importante destacar que, en un año de sequí...",6
1,Hay una gran cantidad de literatura sobre Aust...,6
2,"Los procesos de descentralización, emprendidos...",6
3,Esto puede tener consecuencias sustanciales pa...,6
4,La función de beneficio también incorpora pará...,6


### **2. Separacion de datos**

In [504]:
X, Y = df['Textos_espanol'], df['sdg']
Y = Y.astype(int)
Y.head()

0    6
1    6
2    6
3    6
4    6
Name: sdg, dtype: int32

In [505]:
X.head()

0    Es importante destacar que, en un año de sequí...
1    Hay una gran cantidad de literatura sobre Aust...
2    Los procesos de descentralización, emprendidos...
3    Esto puede tener consecuencias sustanciales pa...
4    La función de beneficio también incorpora pará...
Name: Textos_espanol, dtype: object

In [506]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [507]:
X_train.head()

2370    Lograr el respeto de los derechos humanos por ...
1774    Diseño de proyectos sostenibles de electrifica...
731     En las partes españolas de las cuencas del Due...
271     Garantizar que los arreglos de gobernanza ayud...
1077    Además, a nivel de plantas y unidades, no hay ...
Name: Textos_espanol, dtype: object

### **3. Limpieza de datos**

In [508]:
# Pasar todo el texto a minúsculas
# Quitar signos de puntuación
# Quitar stopwords (palabras y/o expresiones): Por el momento no
# Quitar números (configurable): True
# Quitar acentos (configurable). True

X_train = X_train.apply(lambda x: limpieza_texto(texto=x, quitar_numeros=True, quitar_acentos=True))
X_train.head()

2370    lograr el respeto de los derechos humanos por ...
1774    diseno de proyectos sostenibles de electrifica...
731     en las partes espanolas de las cuencas del due...
271     garantizar que los arreglos de gobernanza ayud...
1077    ademas a nivel de plantas y unidades no hay pr...
Name: Textos_espanol, dtype: object

In [509]:
print(X_train[0])

es importante destacar que en un ano de sequia se espera que disminuyan todos los aportes pero que aumente la demanda de agua de los cultivos y de la vegetacion lo que provocara una reduccion del caudal de los rios que estan conectados hidrologicamente con el acuifero asi como una menor disponibilidad de agua para otros ecosistemas dependientes de las aguas subterraneas estas externalidades espaciales pueden incluir el agotamiento de los cursos de agua y el descenso del nivel freatico local por ejemplo el bombeo continuo del pozo b provocara el agotamiento de los cursos de agua y la reduccion de su caudal


In [510]:
stopwords = lista_stopwords()

def remove_stopwords_from_text(text):
    return remover_stopwords(text, stopwords)

X_train = X_train.apply(lambda x: remove_stopwords_from_text(x))

X_train.head()

2370    lograr respeto derechos humanos parte empresas...
1774    diseno proyectos sostenibles electrificacion r...
731     partes espanolas cuencas duero guadiana respec...
271     garantizar arreglos gobernanza ayuden moviliza...
1077    nivel plantas unidades pruebas diferencias vel...
Name: Textos_espanol, dtype: object

In [511]:
print(X_train[0])

importante destacar ano sequia espera disminuyan aportes aumente demanda agua cultivos vegetacion provocara reduccion caudal rios estan conectados hidrologicamente acuifero menor disponibilidad agua ecosistemas dependientes aguas subterraneas externalidades espaciales pueden incluir agotamiento cursos agua descenso nivel freatico local ejemplo bombeo continuo pozo provocara agotamiento cursos agua reduccion caudal


### **4. Stemming**

In [512]:
stemmer = Stemmer(lenguaje='español')

In [513]:
X_train.head()

X_train = X_train.apply(lambda x: stem_texto(texto=x, stemmer=stemmer))
X_train.head()

2370    logr respet derech human part empres requier h...
1774    disen proyect sosten electrificacion rural con...
731     part espanol cuenc duer guadian respect extrac...
271     garantiz arregl gobern ayud moviliz financi ag...
1077    nivel plant unidad prueb diferent veloc cambi ...
Name: Textos_espanol, dtype: object

### **5. Vectorizacion**

In [514]:
# Vectorizador BOW
v_bow = VectorizadorFrecuencias()

In [515]:
# Hacer una lista, donde cada elemento es un string de la posicion Textos_espanol
textos = [x for x in X_train]

v_bow.ajustar(textos, archivo_salida='vocabulario.txt')

In [516]:
from IPython.display import display

display(v_bow.vocabulario().head(100))

,posición,palabra
0,0,aa
1,1,aan
2,2,aarhus
3,3,aas
4,4,abaj
...,...,...
95,95,acomodacion
96,96,acompan
97,97,acondicion
98,98,aconsej


In [517]:
X_vector = X_train.apply(lambda x: v_bow.vectorizar(textos=x, disperso=False))
X_vector.head()

In [ ]:
# Para entender: Se genera una matriz de una fila con 10146 columnas. La estoy convirtiendo a un array de 10146 elementos.
print(X_vector[0].shape)
print(X_vector[0].ravel())

# Se convierte la matriz en un vector. Se pasa de una matriz de una fila con j columnas a un vector de j elementos.
X_vector_fila = X_vector.apply(lambda x: x.ravel())
X_vector_fila.head()

In [ ]:
print(X_train[0])

import destac ano sequi esper disminu aport aument demand agu cultiv vegetacion provoc reduccion caudal rios estan conect hidrolog acuifer menor disponibil agu ecosistem dependient agu subterran external espacial pued inclu agot curs agu descens nivel freatic local ejempl bombe continu poz provoc agot curs agu reduccion caudal


In [ ]:
v_bow.inversa(X_vector[0])

[array(['acuifer', 'agot', 'agu', 'ano', 'aport', 'aument', 'bombe',
        'caudal', 'conect', 'continu', 'cultiv', 'curs', 'demand',
        'dependient', 'descens', 'destac', 'disminu', 'disponibil',
        'ecosistem', 'ejempl', 'espacial', 'esper', 'estan', 'external',
        'freatic', 'hidrolog', 'import', 'inclu', 'local', 'menor',
        'nivel', 'poz', 'provoc', 'pued', 'reduccion', 'rios', 'sequi',
        'subterran', 'vegetacion'], dtype='<U24')]

### **6. Modelo - Clasificador Naive Bayes**

In [ ]:
# Aplicar todo el preprocesamiento a X_test que se le hizo a X_train
X_test = X_test.apply(lambda x: limpieza_texto(texto=x, quitar_numeros=True, quitar_acentos=True))
X_test = X_test.apply(lambda x: remove_stopwords_from_text(x))
X_test = X_test.apply(lambda x: stem_texto(texto=x, stemmer=stemmer))

# Se convierte el texto a vector
X_test_vector = X_test.apply(lambda x: v_bow.vectorizar(textos=x, disperso=False))
X_test_lista = X_test_vector.apply(lambda x: x.ravel())

X_test_lista.head()

311     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1025    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1587    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2941    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2980    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Textos_espanol, dtype: object

In [ ]:
gnb = GaussianNB()

In [ ]:
gnb.fit(X_vector_fila.values.tolist(), Y_train)

GaussianNB()

In [ ]:
y_pred = gnb.predict(X_test_lista.values.tolist())
y_pred.shape

(600,)

In [ ]:
print("Number of mislabeled points out of a total %d points : %d" 
      % (X_test_lista.shape[0], (Y_test != y_pred).sum()))
print("Porcentaje de acierto: ", (Y_test == y_pred).sum() / Y_test.shape[0])

Number of mislabeled points out of a total 600 points : 59
Porcentaje de acierto:  0.9016666666666666
